In [1]:
%pip install taker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import circuitsvis as cv
import numpy as np
import torch
from taker import Model
from taker.hooks import HookConfig
from datetime import datetime
import json
from os import listdir
from os.path import exists

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import torch

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"../results/{current_time}_agnes_story.jsonl"

In [4]:
m = Model("microsoft/phi-3-mini-4k-instruct", dtype="int8")
m.show_details()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.81s/it]


Loaded model 'microsoft/phi-3-mini-4k-instruct' with int8:
- Added 512 hooks across 32 layers


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.


 - n_layers : 32
 - d_model  : 3072
 - n_heads  : 32
 - d_head   : 96
 - d_mlp    : 8192


In [5]:
prompt    = """Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title."""
# info_gen = m.generate(info_prompt, temperature=0.3, num=300)

#DOUBLE NEWLINE
info_gen = """
\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners.\n\n"""
info_prompt = prompt+info_gen

In [6]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [11]:
if not exists(filename):
    with open(filename, "w") as f:
        pass

neutral_prompt = "Sunday dinner.\n\n"
max_new_tokens = 100
temperature = 0.3
tokens_transferred_num = 5

acts = m.get_midlayer_activations(info_prompt)
orig_token_index = m.get_ids(info_prompt).shape[1] - 1
new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1        



for token_val in range(tokens_transferred_num):
    [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
    for j in range(token_val + 1):
        for layer_index in range(0,32):
            m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index - j, acts["mlp"][0, layer_index, orig_token_index - j])
            m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index - j, acts["attn"][0, layer_index, orig_token_index - j])
    with HiddenPrints():
        for i in range(20):
            output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
            
            data = {
                "temperature": temperature,
                "max_new_tokens": max_new_tokens,
                "model": "microsoft/phi-3-mini-4k-instruct",
                "transplant_layers": (0,32),
                "transferred_token_num": token_val+1,
                "orig_prompt": info_prompt,
                "transplant_prompt": neutral_prompt,
                "output": output[1],
            }

            with open(filename, "a") as file:
                file.write(json.dumps(data) + "\n")

Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out of max 3
Got 304, already saw 0, checking 3 out o

In [6]:
filepath = "../results/2024-08-07_20-30-51_agnes_story.jsonl"
results_df = pd.read_json(filepath, lines=True)
baselinefile = "../results/baseline_data_08-07.jsonl"
base_df = pd.read_json(baselinefile, lines=True)
results_df.head()

,temperature,max_new_tokens,model,transplant_layers,newline_num,orig_prompt,transplant_prompt,output
0,0.3,100,phi-3-mini-4k-instruct,"[0, 32]",2,Write a short blog post about a recipe and the...,\n,**The Art of Crafting a Compelling Narrative**...
1,0.3,100,phi-3-mini-4k-instruct,"[0, 32]",2,Write a short blog post about a recipe and the...,\n,"Now, let's dive into the world of Mathematics,..."
2,0.3,100,phi-3-mini-4k-instruct,"[0, 32]",2,Write a short blog post about a recipe and the...,\n,The key to this recipe is the balance of flavo...
3,0.3,100,phi-3-mini-4k-instruct,"[0, 32]",2,Write a short blog post about a recipe and the...,\n,**Ingredients:**\n\n* 1 large egg\n* 3/4 cup (...
4,0.3,100,phi-3-mini-4k-instruct,"[0, 32]",2,Write a short blog post about a recipe and the...,\n,"This story, while fictional, draws inspiration..."


In [9]:
# sentenceTransformer = SentenceTransformer("all-MiniLM-L6-v2")
sentenceTransformer = SentenceTransformer("all-mpnet-base-v2")

concat_embs = results_df["output"].to_list() + base_df["output"].to_list()
embeddings = sentenceTransformer.encode(concat_embs)
print(embeddings.shape)
results_df["embeddings"] = embeddings[:results_df.shape[0]].tolist()
base_df["embeddings"] = embeddings[results_df.shape[0]:].tolist()
results_df.head()
print(results_df.shape)
print(base_df.shape)

(170, 768)
(120, 9)
(50, 6)


In [15]:
# carry out pca on embeddings using sklearn
reducer = PCA(n_components=2)
embeddings_reduced = reducer.fit_transform(embeddings)
print(embeddings_reduced.shape)

# visualize the embeddings using plotly
import plotly.express as px
import plotly.graph_objects as go

results_df["dim1"] = embeddings_reduced[:results_df.shape[0], 0]
results_df["dim2"] = embeddings_reduced[:results_df.shape[0], 1]
base_df["dim1"] = embeddings_reduced[results_df.shape[0]:, 0]
base_df["dim2"] = embeddings_reduced[results_df.shape[0]:, 1]
baseline_mean_x = base_df["dim1"].mean(axis=0)
baseline_mean_y = base_df["dim2"].mean(axis=0)

results_df["truncated_output"] = results_df["output"].str.slice(0,100)

fig = px.scatter(
    results_df,
    x="dim1",
    y="dim2",
    hover_data=["model", "newline_num"],
    title="PCA of Agnes Story Embeddings",
    color="model",
    symbol="newline_num",
    symbol_sequence = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'pentagon', 'hexagon', 'octagon', 'star'],

)

fig.add_trace(go.Scatter(x=[baseline_mean_x], y=[baseline_mean_y], mode='markers', marker=dict(color='green', size=10), name='Baseline Generation Mean'))


fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=100, xref="container"))

fig.update_traces(
    hovertemplate="<br>".join(
        ["Model: %{customdata[0]}", "Newline Count: %{customdata[1]}"]
    )
)



fig.show()

(170, 2)


In [16]:
pca_df = pd.DataFrame(embeddings_reduced, columns=['PC1', 'PC2'])
pca_df['model'] = results_df['model']
pca_df['newline_num'] = results_df['newline_num']

mean_pca_df = pca_df.groupby(['model', 'newline_num']).mean().reset_index()
print(mean_pca_df.head())

# fig.add_trace(px.scatter(mean_pca_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', 
#                          title='Means of PCA Groups').data[0]):
mean_pca_df['type'] = 'Mean'
pca_df['type'] = 'Original'
combined_df = pd.concat([pca_df, mean_pca_df], ignore_index=True)
# fig = px.scatter(combined_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', title='PCA of Sentence Embeddings')

# Plot mean points
mean_fig = px.scatter(mean_pca_df, x='PC1', y='PC2', color='model', symbol='newline_num', title='Means of PCA Groups', symbol_sequence = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'pentagon', 'hexagon', 'octagon', 'star'])
    
mean_fig.add_trace(go.Scatter(x=[baseline_mean_x], y=[baseline_mean_y], mode='markers', marker=dict(color='green', size=10), name='Baseline Generation Mean'))

mean_fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=100, xref="container"))


mean_fig.show()

                    model  newline_num       PC1       PC2
0              gpt2-large          1.0  0.343078  0.152270
1              gpt2-large          2.0  0.320526  0.184423
2  phi-3-mini-4k-instruct          1.0  0.270573 -0.299580
3  phi-3-mini-4k-instruct          2.0 -0.020827 -0.113707
